<a href="https://colab.research.google.com/github/johnsovo44/RNN-Tutorial/blob/master/NLP_Project(Understanding_RNN_and_LSTM).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install tensorflow

In [1]:
import tensorflow as tf
print(tf.__version__)


1.14.0


# Purpose

I'lll be following along with Jason Brownlee's article on how LTSM is implemented. To differentiate my work from his, he has laid out some extension ideas at the end help with experimentation. I will also use a different unstructured dataset once I have gotten an understanding of his work. 

Link to the article: https://machinelearningmastery.com/text-generation-lstm-recurrent-neural-networks-python-keras/

By the end of this lesson I will know the following:

- Where to download a free corpus of text to further train text generative models: [Project Gutenberg](https://www.gutenberg.org/) (experiments are just limited to text either. You can generate source code, LaTex, HTML or Markdown and more)
- How to frame the problem of text sequences to a recurrent neural network generative model.
- How to develop an LSTM to generate plausible text sequences for a given problem. 

For additional understanding of RNN models please watch this video: [Afr iendly Introduction to Recurrent Neural Networks](https://www.youtube.com/watch?v=UNmqTiOnRfg)

# Approach



1.   Write out code
2. Troubleshoot
2.   Comment out code and what is happening at each step
3. Experiment with extension ideas
4. Insert in own dataset



In [0]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import np_utils

In [8]:
filename = "C:\Users\Vonn\Downloads\wonderland.txt"
raw_text = open(filename, 'r', encoding = 'utf-8').read()
raw_text = raw_text.lower()

SyntaxError: ignored

In [0]:
raw_text = """Project Gutenberg's Alice's Adventures in Wonderland, by Lewis Carroll

This eBook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever.  You may copy it, give it away or
re-use it under the terms of the Project Gutenberg License included
with this eBook or online at www.gutenberg.org


Title: Alice's Adventures in Wonderland

Author: Lewis Carroll

Posting Date: June 25, 2008 [EBook #11]
Release Date: March, 1994
[Last updated: December 20, 2011]

Language: English


*** START OF THIS PROJECT GUTENBERG EBOOK ALICE'S ADVENTURES IN WONDERLAND ***










ALICE'S ADVENTURES IN WONDERLAND

Lewis Carroll

THE MILLENNIUM FULCRUM EDITION 3.0




CHAPTER I. Down the Rabbit-Hole

Alice was beginning to get very tired of sitting by her sister on the
bank, and of having nothing to do: once or twice she had peeped into the
book her sister was reading, but it had no pictures or conversations in
it, 'and what is the use of a book,' thought Alice 'without pictures or
conversations?'

So she was considering in her own mind (as well as she could, for the
hot day made her feel very sleepy and stupid), whether the pleasure
of making a daisy-chain would be worth the trouble of getting up and
picking the daisies, when suddenly a White Rabbit with pink eyes ran
close by her.

There was nothing so VERY remarkable in that; nor did Alice think it so
VERY much out of the way to hear the Rabbit say to itself, 'Oh dear!
Oh dear! I shall be late!' (when she thought it over afterwards, it
occurred to her that she ought to have wondered at this, but at the time
it all seemed quite natural); but when the Rabbit actually TOOK A WATCH
OUT OF ITS WAISTCOAT-POCKET, and looked at it, and then hurried on,
Alice started to her feet, for it flashed across her mind that she had
never before seen a rabbit with either a waistcoat-pocket, or a watch
to take out of it, and burning with curiosity, she ran across the field
after it, and fortunately was just in time to see it pop down a large
rabbit-hole under the hedge.

In another moment down went Alice after it, never once considering how
in the world she was to get out again.

The rabbit-hole went straight on like a tunnel for some way, and then
dipped suddenly down, so suddenly that Alice had not a moment to think
about stopping herself before she found herself falling down a very deep
well.

Either the well was very deep, or she fell very slowly, for she had
plenty of time as she went down to look about her and to wonder what was
going to happen next. First, she tried to look down and make out what
she was coming to, but it was too dark to see anything; then she
looked at the sides of the well, and noticed that they were filled with
cupboards and book-shelves; here and there she saw maps and pictures
hung upon pegs. She took down a jar from one of the shelves as
she passed; it was labelled 'ORANGE MARMALADE', but to her great
disappointment it was empty: she did not like to drop the jar for fear
of killing somebody, so managed to put it into one of the cupboards as
she fell past it."""

raw_text = raw_text.lower()

# this is here because the file will not open right now. Need to troubleshoot
# in second phase. But this is here so I can see what needs troubleshooting. 

In [0]:
chars = sorted(list(set(raw_text)))
char_to_int = dict((c,i) for i, c in enumerate(chars))

In [14]:
char_to_int

{'\n': 0,
 ' ': 1,
 '!': 2,
 '#': 3,
 "'": 4,
 '(': 5,
 ')': 6,
 '*': 7,
 ',': 8,
 '-': 9,
 '.': 10,
 '0': 11,
 '1': 12,
 '2': 13,
 '3': 14,
 '4': 15,
 '5': 16,
 '8': 17,
 '9': 18,
 ':': 19,
 ';': 20,
 '?': 21,
 '[': 22,
 ']': 23,
 'a': 24,
 'b': 25,
 'c': 26,
 'd': 27,
 'e': 28,
 'f': 29,
 'g': 30,
 'h': 31,
 'i': 32,
 'j': 33,
 'k': 34,
 'l': 35,
 'm': 36,
 'n': 37,
 'o': 38,
 'p': 39,
 'q': 40,
 'r': 41,
 's': 42,
 't': 43,
 'u': 44,
 'v': 45,
 'w': 46,
 'x': 47,
 'y': 48}

In [13]:
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("total Vocab: ", n_vocab)

Total Characters:  3084
total Vocab:  49


So N distinct vocab characters are able to produce N characters for the full text. There are obviously characters in here that could be removed in future cleaning but first let's see how they will play a role in the model. 

# Defining the Training Data

(possible area to revisit given flexibility in choosing how to break up the text and exposing it to the network during training. In the tutorial wea re going to split the text in traunches of 100 characters. In the future you could try just the words or sentences.)

A Note on training: "The way it works is each training pattern of the network is comprised of 100 time steps of one character (X) followed by one character output(y)." A window is sliding arcoss 100 characters and then guessing the next output (this goes for every character after the first 100 characters). 100 characters is not standard.

An example of this is the word chapter (from the tutorial):

If we had a 5 character window instead...

X = chapt

y = e

next one...

X = hapte

y = r


In [15]:
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
  seq_in = raw_text[i:i + seq_length]
  seq_out = raw_text[i + seq_length]
  dataX.append([char_to_int[char] for char in seq_in])
  dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

Total Patterns:  2984


The tutorial ended up having the 100 less patterns as they did characters. So if you do this again, make sure that holds true for the full text.

# Prepare for Keras